In [272]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')

In [273]:
melb_df = melb_data.copy()

melb_df.drop(['index','Coordinates'],axis=1,inplace=True)

total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms

diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area

In [274]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
melb_df['Date']

0       2016-12-03
1       2016-02-04
2       2017-03-04
3       2017-03-04
4       2016-06-04
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]

In [275]:
years_sold = melb_df['Date'].dt.year
print(years_sold)
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

0        2016
1        2016
2        2017
3        2017
4        2016
         ... 
13575    2017
13576    2017
13577    2017
13578    2017
13579    2017
Name: Date, Length: 13580, dtype: int64
Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


In [276]:
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: MonthSale, dtype: float64

In [277]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
display(delta_days.dt.days)

0        337
1         34
2        428
3        428
4        155
        ... 
13575    603
13576    603
13577    603
13578    603
13579    603
Name: Date, Length: 13580, dtype: int64

In [278]:
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
display(melb_df['AgeBuilding'])

0         46
1        116
2        117
3         47
4          2
        ... 
13575     36
13576     22
13577     20
13578     97
13579     97
Name: AgeBuilding, Length: 13580, dtype: int64

In [279]:
melb_df = melb_df.drop('YearBuilt', axis=1)

In [280]:
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
#weekend_count = melb_df['WeekdaySale'].value_counts()[5] + melb_df['WeekdaySale'].value_counts()[6]
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]
print(weekend_count)

12822


In [281]:
melb_df['Address'].nunique()

13378

In [282]:
print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])

2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


In [283]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

In [284]:
street_types = melb_df['Address'].apply(get_street_type)
street_types

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

In [285]:
street_types.nunique()

56

In [286]:
street_types.value_counts()

St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Gdns            4
Grn             4
Fairway         4
Mews            4
Crossway        3
Righi           3
Victoria        2
Ridge           2
Crofts          2
Esp             2
Glade           1
Gra             1
Ave             1
Woodland        1
Outlook         1
Hts             1
Highway         1
Athol           1
Summit          1
Grand           1
Res             1
Nook            1
Eyrie           1
Dell            1
East            1
Loop            1
Grange          1
Terrace         1
Cove            1
Qy              1
Corso     

In [287]:
popular_stypes = street_types.value_counts().nlargest(10).index
popular_stypes

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')

In [288]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
melb_df['StreetType'] 

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

In [289]:
melb_df['StreetType'].nunique()

11

In [290]:
melb_df = melb_df.drop('Address', axis=1)

In [291]:
get_weekend = lambda x: 1 if x == 5 or x == 6 else 0

In [292]:
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
melb_df['Weekend']

0        1
1        0
2        1
3        1
4        1
        ..
13575    1
13576    1
13577    1
13578    1
13579    1
Name: Weekend, Length: 13580, dtype: int64

In [293]:
round(melb_df[melb_df['Weekend'] == 1]['Price'].mean())

1081199

In [294]:
popular_sellers = melb_df['SellerG'].value_counts().nlargest(49).index
popular_sellers

Index(['Nelson', 'Jellis', 'hockingstuart', 'Barry', 'Ray', 'Marshall',
       'Buxton', 'Biggin', 'Brad', 'Fletchers', 'Woodards', 'Jas', 'Greg',
       'McGrath', 'Sweeney', 'Noel', 'Miles', 'RT', 'Gary', 'Harcourts',
       'Hodges', 'YPA', 'Stockdale', 'Village', 'Kay', 'Raine', 'Williams',
       'Love', 'Douglas', 'Chisholm', 'RW', 'Rendina', 'HAR', 'O'Brien', 'C21',
       'Collins', 'Cayzer', 'Eview', 'Purplebricks', 'Philip', 'Buckingham',
       'Bells', 'Thomson', 'Nick', 'Alexkarbon', 'McDonald', 'Burnham',
       'Moonee', 'LITTLE'],
      dtype='object')

In [295]:
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_sellers else 'other')
melb_df['SellerG'].value_counts()

Nelson           1565
Jellis           1316
other            1199
hockingstuart    1167
Barry            1011
Ray               701
Marshall          659
Buxton            632
Biggin            393
Brad              342
Woodards          301
Fletchers         301
Jas               243
Greg              239
McGrath           222
Sweeney           216
Noel              205
Miles             196
RT                184
Gary              170
Harcourts         168
Hodges            157
YPA               154
Stockdale         150
Village           125
Kay               119
Raine             116
Williams          111
Love              109
Douglas            97
Chisholm           77
RW                 70
Rendina            66
HAR                62
O'Brien            61
C21                57
Collins            56
Cayzer             52
Eview              51
Purplebricks       51
Philip             48
Buckingham         46
Bells              44
Thomson            42
Nick               40
Alexkarbon

In [296]:
round(melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min() / melb_df[melb_df['SellerG'] == 'other']['Price'].min(), 1)

1.3

In [297]:
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtype) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts)

,Column_Name,Num_Unique,Type
0,Weekend,2,int64
1,Type,3,object
2,Method,5,object
3,WeekdaySale,5,int64
4,Regionname,8,object
5,Bathroom,9,int64
6,Rooms,9,int64
7,Car,11,int64
8,StreetType,11,object
9,Bedroom,12,int64


In [298]:
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  object        
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  object        
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  object        
 5   SellerG          13580 non-null  object        
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  objec

In [299]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  object        
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

None

In [300]:
print(melb_df['Regionname'].cat.categories)

Index(['Eastern Metropolitan', 'Eastern Victoria', 'Northern Metropolitan',
       'Northern Victoria', 'South-Eastern Metropolitan',
       'Southern Metropolitan', 'Western Metropolitan', 'Western Victoria'],
      dtype='object')


In [301]:
display(melb_df['Regionname'].cat.codes)

0        2
1        2
2        2
3        2
4        2
        ..
13575    4
13576    6
13577    6
13578    6
13579    6
Length: 13580, dtype: int8

In [302]:
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
display(melb_df['Type'])

0        house
1        house
2        house
3        house
4        house
         ...  
13575    house
13576    house
13577    house
13578    house
13579    house
Name: Type, Length: 13580, dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

In [303]:
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

0     unit
1    house
2     flat
3     flat
4    house
dtype: category
Categories (4, object): ['house', 'townhouse', 'unit', 'flat']

In [304]:
popular_suburbs = melb_df['Suburb'].value_counts().nlargest(119).index
popular_suburbs

Index(['Reservoir', 'Richmond', 'Bentleigh East', 'Preston', 'Brunswick',
       'Essendon', 'South Yarra', 'Glen Iris', 'Hawthorn', 'Coburg',
       ...
       'Albion', 'Hoppers Crossing', 'Sunbury', 'Greensborough', 'Bundoora',
       'Hughesdale', 'Chadstone', 'Caulfield North', 'Mont Albert',
       'Alphington'],
      dtype='object', length=119)

In [309]:
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_suburbs else 'other')
melb_df['Suburb'].nunique()

120

In [311]:
melb_df['Suburb'] = melb_df['Suburb'].astype('category')
melb_df['Suburb']

0          Abbotsford
1          Abbotsford
2          Abbotsford
3          Abbotsford
4          Abbotsford
             ...     
13575           other
13576    Williamstown
13577    Williamstown
13578    Williamstown
13579      Yarraville
Name: Suburb, Length: 13580, dtype: category
Categories (120, object): ['Abbotsford', 'Aberfeldie', 'Airport West', 'Albert Park', ..., 'Williamstown', 'Windsor', 'Yarraville', 'other']

In [312]:
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ